### This notebook contains NHS data analysis and training and includes following

<b> 1) Training with fasttext features<br>
<b> 2) gridserach on svc and fasttext<br>
<b> 3) Training with tfidf features <br>
<b> 4) gridserach on svc and tfidf features<br>
<b> 5) Clustering to remove duplicates<br>
<b> 6) Removing false positive and conflicting train data<br>
<b> 7) Reducing categories<br>


In [1]:
import pandas as pd
import sklearn
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
import string
from textblob import Word
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop = stopwords.words('english')
from sklearn.svm import SVC

Loading training data till april and testing it on it may<br>

In [2]:
path_org=r"C:\Users\annautiyal\Desktop\input_file\Train data tipp Arpil.xlsx"
df_train=pd.read_excel(path_org)
path=r"C:\Users\annautiyal\Desktop\input_file\NHS_MAY1_with_33queryType.xlsx"
df_test=pd.read_excel(path)

In [3]:
df_train.shape,df_test.shape

((21376, 12), (2611, 12))

First we will train it on fasttext model

In [4]:
"""
process_data function takes input data frame and return list of list containing each training sample after removing
stopwords,punctutaions and tokenization
This function prepare data for fasttext training model
"""
def process_data(data):
    #data = data.astype(str).apply(lambda x: x.str.lower())
    data = data.apply(lambda x: x.str.lower())
    processdata = data
    processdata['extracted_problem']= processdata['extracted_problem'].apply(lambda x: ' '.join(x for x in x.split() if x not in string.punctuation))
    processdata['extracted_problem']= processdata['extracted_problem'].str.replace('[^\w\s]','')
    processdata['extracted_problem']= processdata['extracted_problem'].apply(lambda x: ' '.join(x for x in x.split() if  not x.isdigit()))
    processdata['extracted_problem']= processdata['extracted_problem'].apply(lambda x:' '.join(x for x in x.split() if not x in stop))
    processdata['extracted_problem']= processdata['extracted_problem'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
    processdata['extracted_problem']= processdata['extracted_problem'].apply(word_tokenize)
    #rint(processdata['extracted_problem'])
    bigger_list=[]
    for i in processdata['extracted_problem']:
         bigger_list.append(i)
    return bigger_list
    

In [5]:
train_list=process_data(df_train.astype(str))
train_list[0] ##first training example

['summary', 'cmrs', 'stuck', 'sysadmin', 'error']

In [6]:
"""
ft function will train word embedding using skipgram algo and return 100 dimension vector
"""
from gensim.models import FastText as ft
from gensim.models import Word2Vec
model = ft(train_list,sg=1, size=100, window=16, min_count=1, iter=50,cbow_mean=0,sorted_vocab=0,min_n=2,max_n=6,word_ngrams=2)

C:\Users\annautiyal\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [7]:
"""
saving and loading the model
"""
# model.save("word2vec.model")
#model=Word2Vec.load("word2vec.model")

'\nsaving and loading the model\n'

In [8]:
"""
preparing data to convert them into embeddings  and replacing empty 'extracted_problem' with 'Not Extracted'
"""
train_problem_description = process_data(df_train.astype(str))
train_description1=[]
for i in train_problem_description:
    strm=' '.join(i)
    if len(strm.strip())>0:  
        train_description1.append(strm)
    else:
        train_description1.append('Not Extracted')

test_problem_description = process_data(df_test.astype(str))
test_description1=[]
for i in test_problem_description:
    strm=' '.join(i)
    if len(strm.strip())>0:  
        test_description1.append(strm)
    else:
        test_description1.append('Not Extracted')        

In [9]:
len(train_description1),len(test_description1)

(21376, 2611)

In [10]:
train_embedding=model.wv[train_description1]
test_embedding=model.wv[test_description1]

In [11]:
train_embedding.shape,test_embedding.shape

((21376, 100), (2611, 100))

In [12]:
"""
using grid serach to find best parameters for svc, it took 4 hr to find best parameters
"""
# import sklearn
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import classification_report, confusion_matrix
# param_grid = {'C':[1,10,100,1000],'gamma':[1,0.1,0.001,0.0001], 'kernel':['linear','rbf','poly']}
# grid = GridSearchCV(SVC(),param_grid,refit = True, verbose=2 ,n_jobs=2)

'\nusing grid serach to find best parameters for svc, it took 4 hr to find best parameters\n'

In [13]:
"""
now using those parameters in svc to train the data for parameters C=10,kernel='poly',gamma=.1
"""
modelSVC = SVC(C=10,kernel='poly',gamma=.1)
modelSVC.fit(train_embedding,df_train['Call type:Query type'].astype(str))

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [14]:
"""
Prediction Score 
"""
modelSVC.score(test_embedding,df_test['Call type:Query type'])

0.729988510149368

In [15]:
"""
For tuning parameters for fasttext model we will train different models and then generate accuracy with them
it will take 4 hr to  train all models using 3 workers

parameter used for tuning
wordNgrams,learningrate,dimensionality,epoch
"""
## tuning fasttext
# import itertools
# import os

# li_param=[]
# wordNgrams=[1,2]
# lr=[0.05,0.025,0.25]
# dim=[100,200]
# epoch=[50,100]
# li_param.append(wordNgrams)
# li_param.append(lr)
# li_param.append(dim)
# li_param.append(epoch)

# for element in itertools.product(*li_param):
#     print(element)
#     model = ft(big_list,sg=1, size=element[2], alpha=element[1], min_count=1, iter=element[3],cbow_mean=0,sorted_vocab=0,min_n=2,max_n=6,word_ngrams=element[0],workers=3,batch_words=1000)
# #     workers=4,batch_words=1000
#     nli=[str(i)+'-' for i in list(element)]
#     path='fasttextmodels/'+''.join(nli)
#     os.mkdir(path)
#     model.save(path+'/'+'word2vec.model')

'\nFor tuning parameters for fasttext model we will train different models and then generate accuracy with them\nit will take 4 hr to  train all models using 3 workers\n\nparameter used for tuning\nwordNgrams,learningrate,dimensionality,epoch\n'

In [16]:
"""
using tfidf to train the svc model and using grid best parameters for tfidf feature C=1,kernel='linear',gamma=.1
"""
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 3))
X = vectorizer.fit_transform(df_train['extracted_problem'].values.astype('U'))
y_test= vectorizer.transform(df_test['extracted_problem'].values.astype('U'))

In [17]:
X.shape,y_test.shape

((21376, 450476), (2611, 450476))

In [18]:
modelSVC = SVC(C=1,kernel='linear',gamma=.1)
modelSVC.fit(X,df_train['Call type:Query type'].astype(str))

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [19]:
x_tra= vectorizer.transform(df_train['extracted_problem'].values.astype('U'))
x_train=modelSVC.predict(x_tra)
x_tes= vectorizer.transform(df_test['extracted_problem'].values.astype('U'))
x_test=modelSVC.predict(x_tes)
df_train['out_pred']=x_train
df_test['out_pred']=x_test

In [20]:
"""testing and training accuracy"""
from sklearn.metrics import accuracy_score
test_sc=modelSVC.score(y_test,df_test['Call type:Query type'])
train_sc=accuracy_score(df_train['Call type:Query type'], x_train)
train_sc,test_sc

(0.9345527694610778, 0.7606281118345462)

In [21]:
### displaying category wise accuracy
from sklearn.metrics import accuracy_score

grp=df_test.groupby(['Call type:Query type'])
li_grp=df_test['Call type:Query type'].unique()
li_acc=[]
li_str=[]
li_item=[]
li_len=[]
for item in li_grp:
    grp1=grp.get_group(item)
    score=accuracy_score(grp1['Call type:Query type'],grp1['out_pred'])
    li_acc.append(score)
    li_item.append(item)
    li_len.append(grp1.shape[0])
    strm=item+'    accuracy_score %'+str(score) +'  length-:'+str(grp1.shape[0])
    li_str.append(strm)
li_acc, li_str = zip(*sorted(zip(li_acc, li_str)))  
for item in li_str:
    print(item)

Azure:Fault/Query    accuracy_score %0.0  length-:2
Business Intelligence:BI Hierarchy    accuracy_score %0.0  length-:2
CLOUD Domain:Password Reset Request    accuracy_score %0.0  length-:4
CRM System:User End Date    accuracy_score %0.0  length-:16
Citrix:CMR Workflow Error    accuracy_score %0.0  length-:2
Network:External Website Access Issues    accuracy_score %0.0  length-:2
Oracle:Non PO Rules Request    accuracy_score %0.0  length-:5
Oracle:User Request Bulk Change    accuracy_score %0.0  length-:14
RPA-Kapow:Fault/Query    accuracy_score %0.0  length-:1
RPA-UiPath:Fault/Query    accuracy_score %0.0  length-:4
RPA-UiPath:R12 Login Assistance    accuracy_score %0.0  length-:1
Web Form:Performance Issue    accuracy_score %0.0  length-:3
Oracle:CMR Workflow Request    accuracy_score %0.05263157894736842  length-:19
Oracle:Accounts Payable Query    accuracy_score %0.125  length-:16
Oracle:Log In Problem Individual    accuracy_score %0.125  length-:32
NHS Mail:Fault/Query    accurac

In [22]:
"""
using clustering to remove  duplicates
"""
li_lab=pd.unique(df_train['Call type:Query type'])

In [23]:
## unique categories
li_lab

array(['Oracle:CMR Workflow Error', 'AURORA:Password Reset Request',
       'NHS Mail:Password Reset', 'NHS Mail:Shared Mailbox',
       'Oracle:User End Date', 'Oracle:User Amend',
       'Kofax KTM:Performance Issue', 'Oracle:User New Setup',
       'Business Intelligence:BI Report Query',
       'Oracle:Password Reset Request', 'Oracle:Process Query',
       'Oracle:R12 Login Assistance', 'Oracle:Workflow Request',
       'Oracle:Reports Query', 'NHS Mail:Fault/Query',
       'Outlook:Fault/Query', 'Skype for Business:Fault/Query',
       'Kofax MailManager:Fault/Query', 'Oracle:CMR Cancellation',
       'Oracle:Workflow Rule', 'Manual Payments:Fault/Query',
       'Oracle:JRE/JInitiator Query', 'Oracle:Log In Problem Individual',
       'Web Form:Performance Issue', 'Oracle:General Ledger Query',
       'vQSM:Operational Query', 'Outlook:Archive',
       'PC:Performance Issue', 'Oracle:CMR Workflow Request',
       'Oracle:Accounts Payable Query', 'IT Intranet:Password Reset',
    

In [24]:
"""
cluster_data function takes data of each category and return cluster labels for each samples
"""
from sklearn.cluster import KMeans
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
def cluster_data(li):
    vectorizer = TfidfVectorizer(max_df=0.5,ngram_range=(1, 3))
    X = vectorizer.fit_transform(li.values.astype('U'))
    kmeans = KMeans(n_clusters=6, random_state=0).fit(X)
    return kmeans.labels_

In [25]:
count=0
grp=df_train.groupby('Call type:Query type') ##group dataframe based on call type query type
for item in li_lab[:30]:
#     print(count)
    nli=grp.get_group(item)['extracted_problem']
    label_li=cluster_data(nli)
    label_li=[str(i)+'-'+item for i in list(label_li)]
    li_index=list(grp.get_group(item)['extracted_problem'].index.values)
    for j in range(len(li_index)):
        df_train.at[li_index[j], 'predicted_value'] = label_li[j]
    count+=1

In [26]:
"""
save cluster outputs
"""
# df_train.to_excel('cluster_out.xlsx')

'\nsave cluster outputs\n'

In [27]:
## sample cluster outputs
df_train['predicted_value'].head()

0        0-Oracle:CMR Workflow Error
1        0-Oracle:CMR Workflow Error
2    0-AURORA:Password Reset Request
3          2-NHS Mail:Password Reset
4          2-NHS Mail:Password Reset
Name: predicted_value, dtype: object

#### After clustering we have removed duplictaes from train data manually

In [28]:
### importing filter data
path3=r"C:\Users\annautiyal\Desktop\outputs\cluster_out.xlsx"
df_clu_tr=pd.read_excel(path3)

In [29]:
df_clu_tr.shape,df_test.shape

((11307, 14), (2611, 13))

Now training on filter data

In [30]:
vectorizer = TfidfVectorizer(ngram_range=(1, 3))
X = vectorizer.fit_transform(df_clu_tr['extracted_problem'].values.astype('U'))
y_test= vectorizer.transform(df_test['extracted_problem'].values.astype('U'))

modelSVC = SVC(C=1,kernel='linear',gamma=.1)
modelSVC.fit(X,df_clu_tr['Call type:Query type'].astype(str))

x_tra= vectorizer.transform(df_clu_tr['extracted_problem'].values.astype('U'))
x_train=modelSVC.predict(x_tra)
x_tes= vectorizer.transform(df_test['extracted_problem'].values.astype('U'))
x_test=modelSVC.predict(x_tes)
df_clu_tr['out_pred']=x_train
df_test['out_pred']=x_test

In [31]:
"""testing and training accuracy"""
from sklearn.metrics import accuracy_score
test_sc=modelSVC.score(y_test,df_test['Call type:Query type'])
train_sc=accuracy_score(df_clu_tr['Call type:Query type'], x_train)
train_sc,test_sc

(0.9210223755195897, 0.6208349291459211)

In [32]:
### displaying category wise accuracy
from sklearn.metrics import accuracy_score

grp=df_test.groupby(['Call type:Query type'])
li_grp=df_test['Call type:Query type'].unique()
li_acc=[]
li_str=[]
li_item=[]
li_len=[]
for item in li_grp:
    grp1=grp.get_group(item)
    score=accuracy_score(grp1['Call type:Query type'],grp1['out_pred'])
    li_acc.append(score)
    li_item.append(item)
    li_len.append(grp1.shape[0])
    strm=item+'    accuracy_score %'+str(score) +'  length-:'+str(grp1.shape[0])
    li_str.append(strm)
li_acc, li_str = zip(*sorted(zip(li_acc, li_str)))  
for item in li_str:
    print(item)

Azure:Fault/Query    accuracy_score %0.0  length-:2
Business Intelligence:BI Hierarchy    accuracy_score %0.0  length-:2
CLOUD Domain:Password Reset Request    accuracy_score %0.0  length-:4
CRM System:User End Date    accuracy_score %0.0  length-:16
Citrix:CMR Workflow Error    accuracy_score %0.0  length-:2
Network:External Website Access Issues    accuracy_score %0.0  length-:2
Oracle:Non PO Rules Request    accuracy_score %0.0  length-:5
Oracle:User Request Bulk Change    accuracy_score %0.0  length-:14
RPA-Kapow:Fault/Query    accuracy_score %0.0  length-:1
RPA-UiPath:Fault/Query    accuracy_score %0.0  length-:4
RPA-UiPath:R12 Login Assistance    accuracy_score %0.0  length-:1
Web Form:Performance Issue    accuracy_score %0.0  length-:3
Oracle:Accounts Payable Query    accuracy_score %0.125  length-:16
vQSM:Password Reset Request    accuracy_score %0.2  length-:5
NHS Mail:Fault/Query    accuracy_score %0.22413793103448276  length-:58
Skype for Business:Fault/Query    accuracy_sco

Now removing false positive  cases for different categories from training data<br>
And removing  16 categories for which data is less and for which training data is conflicting

In [33]:
"""
reading filtered file 
"""
path4=r"C:\Users\annautiyal\Desktop\outputs\filter_out_lat_sh.xlsx"

In [34]:
li_drop=['RPA-UiPath:R12 Login Assistance',
'Kofax Analytics:Performance Issue',
'Microsoft Applications:Process Query',
'Citrix:CMR Workflow Error',
'Kofax Analytics:Password Reset',
'Business Intelligence:BI Hierarchy',
'CLOUD Domain:Password Reset Request',
'Kofax MailManager:Performance Issue',
'Web Form:Performance Issue',
'Kofax Analytics:Fault/Query',
'Manual Payments:Fault/Query',
'SharePoint:Fault/Query',
'Oracle:R12 Login Assistance',
'Oracle:Log In Problem Individual',
'NHS Mail:Fault/Query',
'Oracle:Password Reset Request'
]

In [35]:
df_clu_fp=pd.read_excel(path4,'Sheet1')

In [36]:
for item in li_drop:
    # Get names of indexes for which column Age has value 30
    indexNames = df_test[ df_test['Call type:Query type'] == item ].index 
    # Delete these row indexes from dataFrame
    df_test.drop(indexNames , inplace=True)

In [37]:
df_clu_fp.shape,df_test.shape

((9543, 13), (2213, 13))

In [38]:
vectorizer = TfidfVectorizer(ngram_range=(1, 3))
X = vectorizer.fit_transform(df_clu_fp['extracted_problem'].values.astype('U'))
y_test= vectorizer.transform(df_test['extracted_problem'].values.astype('U'))

modelSVC = SVC(C=1,kernel='linear',gamma=.1)
modelSVC.fit(X,df_clu_fp['Call type:Query type'].astype(str))

x_tra= vectorizer.transform(df_clu_fp['extracted_problem'].values.astype('U'))
x_train=modelSVC.predict(x_tra)
x_tes= vectorizer.transform(df_test['extracted_problem'].values.astype('U'))
x_test=modelSVC.predict(x_tes)
df_clu_fp['out_pred']=x_train
df_test['out_pred']=x_test

In [39]:
"""testing and training accuracy"""
from sklearn.metrics import accuracy_score
test_sc=modelSVC.score(y_test,df_test['Call type:Query type'])
train_sc=accuracy_score(df_clu_fp['Call type:Query type'], x_train)
train_sc,test_sc

(0.9458241643089176, 0.789877993673746)

In [40]:
### displaying category wise accuracy
from sklearn.metrics import accuracy_score

grp=df_test.groupby(['Call type:Query type'])
li_grp=df_test['Call type:Query type'].unique()
li_acc=[]
li_str=[]
li_item=[]
li_len=[]
for item in li_grp:
    grp1=grp.get_group(item)
    score=accuracy_score(grp1['Call type:Query type'],grp1['out_pred'])
    li_acc.append(score)
    li_item.append(item)
    li_len.append(grp1.shape[0])
    strm=item+'    accuracy_score %'+str(score) +'  length-:'+str(grp1.shape[0])
    li_str.append(strm)
li_acc, li_str = zip(*sorted(zip(li_acc, li_str)))  
for item in li_str:
    print(item)

Azure:Fault/Query    accuracy_score %0.0  length-:2
CRM System:User End Date    accuracy_score %0.0  length-:16
Network:External Website Access Issues    accuracy_score %0.0  length-:2
Oracle:Non PO Rules Request    accuracy_score %0.0  length-:5
Oracle:User Request Bulk Change    accuracy_score %0.0  length-:14
RPA-Kapow:Fault/Query    accuracy_score %0.0  length-:1
RPA-UiPath:Fault/Query    accuracy_score %0.0  length-:4
Oracle:Accounts Payable Query    accuracy_score %0.125  length-:16
Skype for Business:Fault/Query    accuracy_score %0.25  length-:4
CRM System:Password Reset Request    accuracy_score %0.2857142857142857  length-:7
Business Intelligence:BI Login Issue - Individual    accuracy_score %0.3333333333333333  length-:18
Oracle:CMR Workflow Request    accuracy_score %0.3684210526315789  length-:19
Kofax KTM:Performance Issue    accuracy_score %0.4  length-:5
Kofax MailManager:Fault/Query    accuracy_score %0.4  length-:5
Oracle:JRE/JInitiator Query    accuracy_score %0.4  l

In [41]:
len(df_test)

2213

In [42]:
df_test.to_excel('test_may.xlsx')